## 处理训练文本

In [1]:
#读取训练文本
import pandas as pd
df = pd.read_csv('./data/YNU.EDU2018-ScenicWord/train_first.csv')
df.head(5)

,Id,Discuss,Score
0,201e8bf2-77a2-3a98-9fcf-4ce03914e712,好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的,5
1,f4d51947-eac4-3005-9d3c-2f32d6068a2d,新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！,4
2,74aa7ae4-03a4-394c-bee0-5702d3a3082a,庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去,4
3,099661c2-4360-3c49-a2fe-8c783764f7db,个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...,5
4,97ca672d-e558-3542-ba7b-ee719bba1bab,迪斯尼一日游,5


In [2]:
df.describe()

,Score
count,100000.000000
mean,4.470010
std,0.752134
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


## 文本分词

In [3]:
import jieba
import re
#对pd.DataFramd分词
def fenci(data):
    seg_list = []
    addlist = ['\n',',',"\n"]
    
    for d in data.iterrows():
        sentence = d[1].Discuss.decode('utf8')
        text = re.sub(u'[a-zA-Z0-9.。：:;；，,)）(（！!?？”“\"\<\>《》/]','',sentence).split()
        print text
        text1 = []
        for t in text:
            seg2 = []
            seg = list(jieba.cut(t))
            for s in seg:
                if s not in addlist and len(s)>1:
                    seg2.append(s)
        seg_list.append(seg2)
    
        #停用词，无效符号
        #print','.join(seg_list)
    return seg_list

In [4]:
#对字符串数据content分词
def fenci_content(content):
    seg_list = []
    addlist = ['\n',',',"\n"]
    text = re.sub(u'[a-zA-Z0-9.。：:;；，,)）(（！!?？”“\"\<\>《》/]','',content).split()  
    for t in text:
        seg = list(jieba.cut(t))
        for s in seg:
            if s not in addlist and len(s)>1:
                seg_list.append(s)
    return seg_list

In [5]:
from sklearn.cross_validation import train_test_split
import numpy
#将数据划分成训练数据和测试数据
train,test = train_test_split(df)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
test.describe()

,Score
count,25000.000000
mean,4.477280
std,0.748376
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [7]:
train.describe()

,Score
count,75000.000000
mean,4.467587
std,0.753373
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


## 标签化

In [8]:
#对pd.DataFrame每条记录评价文本分词并打标签
import gensim 
import collections
import multiprocessing
def tokenize1(data,tokens_only=False):
    data_len = len(data)
    corpus = []
    for i in range(data_len):
        d = data.iloc[i]
        content = d.Discuss.decode('utf8')
        if tokens_only:
            corpus.append(fenci_content(content))
        else:
            #对训练数据打标签
            corpus.append(gensim.models.doc2vec.TaggedDocument(fenci_content(content),[i]))
    return corpus
    
            

In [9]:
train_corpus = list(tokenize1(train))
test_corpus = list(tokenize1(test,tokens_only=True))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.528 seconds.
Prefix dict has been built succesfully.


## 构建Doc2vec模型

In [15]:
#实例化Doc2Vec对象
cores = multiprocessing.cpu_count()
models = [
    #PV-DBOW
    gensim.models.doc2vec.Doc2Vec(dm=0,dbow_words=1,size=80,min_count=2,iter=55,workers=cores,window=8),
    #PV-DM w/average
    gensim.models.doc2vec.Doc2Vec(dm=1,dm_mean=1,size=80,min_count=2,iter=55,workers=cores,window=8),
]

In [16]:
#build a vocabulary
models[0].build_vocab(train_corpus)
models[1].reset_from(models[0])

In [17]:
#模型训练
for model in models:
    %time model.train(train_corpus,total_examples=model.corpus_count,epochs=model.iter)

CPU times: user 21min 45s, sys: 1min 16s, total: 23min 1s
Wall time: 10min 59s
CPU times: user 7min 51s, sys: 1min 12s, total: 9min 3s
Wall time: 6min 42s


In [26]:
#模型保存
models[0].save('./PV_DBOW_model')
models[1].save('./PV_DM_model')

## doc vec生成

In [ ]:
#新输入文本（词的list形式）,生成文本向量
#model.infer_vector(['only','you','can','prevent','forecast','fires'])
model.infer_vector(test_corpus[1]).shape

## Assessing model模型评估

In [19]:
#评估模型，打印模型self_similarity
from __future__ import division
def access_model(model,train_corpus):
    ranks = []
    second_ranks = []
    for doc_id in range(len(train_corpus)):
        inferred_vector = model.infer_vector(train_corpus[doc_id].words)
        sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
        rank = [docid for docid, sim in sims].index(doc_id) #查找doc_id的索引位置 
        #rank =0时，表示doc_id的inferred_vector最相近的向量使其本身
        ranks.append(rank)

        second_ranks.append(sims[1])
    m = collections.Counter(ranks)
    sum1 = m[0]
    sum2 = sum(m.values())
    print 'most similar is itself:',sum1
    print 'similar doc per model %s'%(sum1/sum2)

In [21]:
%time access_model(models[0],train_corpus)
#%time access_model(model[1],train_corpus)

most similar is itself: 56732
similar doc per model 0.756426666667
CPU times: user 7h 18min 33s, sys: 13h 8min 32s, total: 20h 27min 5s
Wall time: 8h 23min 44s


In [ ]:
%time access_model(models[1],train_corpus)